<a href="https://colab.research.google.com/github/kaarelkaarelson/AA/blob/master/week4/arabic_w4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4: Arabic

Span-based question answering

In [1]:
# Google Colab setup
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/")

Mounted at /content/drive


In [26]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install evaluate

from transformers import pipeline, BertTokenizer, AutoConfig, Trainer, TrainingArguments, BertForQuestionAnswering, AutoTokenizer, DataCollatorWithPadding,Trainer, AutoModelForQuestionAnswering
from datasets import load_dataset, Dataset
import pandas as pd
import string
import nltk
import math
import statistics
from huggingface_hub import notebook_login
import pickle
from tqdm import tqdm
import torch
import concurrent.futures
from sklearn.metrics import f1_score
import numpy as np


nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
notebook_login()

In [5]:
full_dataset = load_dataset("copenlu/answerable_tydiqa")
full_dataset = full_dataset.rename_column("question_text", "question")
full_dataset = full_dataset.rename_column("document_plaintext", "context")
full_dataset = full_dataset.rename_column("annotations", "answer")
train = full_dataset["train"]
validation = full_dataset["validation"]

# Filter Arabic
train = train.filter(lambda x: x["language"]=="arabic").select_columns(["question", "context", "answer"])
validation = validation.filter(lambda x: x["language"]=="arabic").select_columns(["question", "context", "answer"])
validation_add_ids = pd.DataFrame(validation).reset_index().rename(columns={"index": "id"})
validation = Dataset.from_pandas(validation_add_ids)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/116067 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13325 [00:00<?, ? examples/s]

Filter:   0%|          | 0/116067 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13325 [00:00<?, ? examples/s]

## Testing

In [7]:
# Load all the baseline and finetuned language models.

arabert = pipeline("question-answering", model="aubmindlab/bert-large-arabertv2")
araelectra = pipeline("question-answering", model="ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA")
multilingual_bert = pipeline("question-answering", model="bert-base-multilingual-cased")
finetuned_arabert = pipeline("question-answering", model="kaarelkaarelson/finetuned-bert-base-arabertv2")
finetuned_araelectra = pipeline("question-answering", model="kaarelkaarelson/finetuned-AraElectra-Arabic-SQuADv2-QA")
finetuned_multilingual_bert = pipeline("question-answering", model="kaarelkaarelson/finetuned_bert-base-multilingual-cased")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/bert-large-arabertv2 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Retrieve data structures from pickle files and assign them to corresponding variables
with open('w3_preds_arabert.pkl', 'rb') as file:
    preds_arabert = pickle.load(file)

with open('w3_preds_araelectra.pkl', 'rb') as file:
    preds_araelectra = pickle.load(file)

with open('w3_preds_multilingual_bert.pkl', 'rb') as file:
    preds_multilingual_bert = pickle.load(file)

with open('w3_preds_finetuned_arabert.pkl', 'rb') as file:
    preds_finetuned_arabert = pickle.load(file)

with open('w3_preds_finetuned_araelectra.pkl', 'rb') as file:
    preds_finetuned_araelectra = pickle.load(file)

with open('w3_preds_finetuned_multilingual_bert.pkl', 'rb') as file:
    preds_finetuned_multilingual_bert = pickle.load(file)

In [17]:
def find_best_threshold(preds, true_labels):
    step = 0.0005
    thresholds = [round(step*i, 4) for i in range(2000)]

    best_threshold = 0
    best_f1 = 0

    for threshold in thresholds:
        binary_preds = [1 if pred['score'] > threshold else 0 for pred in preds]
        f1 = f1_score(true_labels, binary_preds)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold

In [12]:
def transform_to_df(preds, scores, threshold):
  true_labels = []
  labels_pred = []

  for i in range(len(validation)):
    if validation[i]["answer"]["answer_start"]==[-1]:
      true_labels.append(False)
    else:
      true_labels.append(True)

    labels_pred.append(1 if preds[i]['score'] > threshold else 0)

  df = pd.DataFrame({"Score":scores, "Class":true_labels, "Prediction":labels_pred})

  return df

In [13]:
def calculate_f1_score_with_tokens(predicted_start, predicted_end, true_start, true_end):
    # Calculate overlap between predicted and true answer tokens
    common_tokens = set(range(predicted_start, predicted_end + 1)) & set(range(true_start, true_end + 1))

    # Calculate precision, recall, and F1 score
    precision = len(common_tokens) / (predicted_end - predicted_start + 1) if predicted_end >= predicted_start else 0
    recall = len(common_tokens) / (true_end - true_start + 1) if true_end >= true_start else 0
    f1 = (2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return f1

In [18]:
true_labels = [0 if validation[i]["answer"]["answer_start"] == [-1] else 1 for i in range(len(validation))]

scores_arabert = [pred['score'] for pred in preds_arabert]
scores_finetuned_arabert = [pred['score'] for pred in preds_finetuned_arabert]
scores_araelectra= [pred['score'] for pred in preds_araelectra]
scores_finetuned_araelectra= [pred['score'] for pred in preds_finetuned_araelectra]
scores_multilingual_bert = [pred['score'] for pred in preds_multilingual_bert]
scores_finetuned_multilingual_bert= [pred['score'] for pred in preds_finetuned_multilingual_bert]

threshold_arabert = find_best_threshold(preds_arabert, true_labels)
threshold_finetuned_arabert = find_best_threshold(preds_finetuned_arabert, true_labels)
threshold_araelectra = find_best_threshold(preds_araelectra, true_labels)
threshold_finetuned_araelectra = find_best_threshold(preds_finetuned_araelectra, true_labels)
threshold_multilingual_bert = find_best_threshold(preds_multilingual_bert, true_labels)
threshold_finetuned_multilingual_bert = find_best_threshold(preds_finetuned_multilingual_bert, true_labels)

df_arabert = transform_to_df(preds_arabert, scores_arabert, threshold_arabert)
df_finetuned_arabert = transform_to_df(preds_finetuned_arabert, scores_finetuned_arabert, threshold_finetuned_arabert)
df_araelectra = transform_to_df(preds_araelectra, scores_araelectra, threshold_araelectra)
df_finetuned_araelectra = transform_to_df(preds_finetuned_araelectra, scores_finetuned_araelectra, threshold_finetuned_araelectra)
df_multilingual_bert = transform_to_df(preds_multilingual_bert, scores_multilingual_bert, threshold_multilingual_bert)
df_finetuned_multilingual_bert = transform_to_df(preds_finetuned_multilingual_bert, scores_finetuned_multilingual_bert, threshold_finetuned_multilingual_bert)

In [37]:
from evaluate import load

# preds = preds_finetuned_arabert
# preds = preds_finetuned_araelectra
preds = preds_finetuned_multilingual_bert

squad_v2_metric = load("squad_v2")
valid = []
res = []
for i in range(len(preds)):
  res.append({})
  res[i]["id"] = str(validation[i]["id"])
  res[i]["no_answer_probability"] = 1 - preds[i]["score"]
  res[i]["no_answer_probability"] = np.float32(res[i]["no_answer_probability"])
  res[i]["prediction_text"] = preds[i]["answer"]
  tmp = {}
  tmp["answers"] = validation[i]["answer"]
  tmp["id"] = str(validation[i]["id"])
  tmp["answers"]["text"] = tmp["answers"]["answer_text"]
  if(tmp["answers"]["answer_start"]==[-1]):
    tmp["answers"]["answer_start"] = []
    tmp["answers"]["text"] = []
  tmp["answers"].pop("answer_text")
  tmp["answers"]["answer_start"] = np.int32(tmp["answers"]["answer_start"])

  valid.append(tmp)

In [38]:
metrics = squad_v2_metric.compute(predictions=res, references=valid)

In [39]:
metrics

{'exact': 24.55310199789695,
 'f1': 34.051352153694076,
 'total': 1902,
 'HasAns_exact': 49.1062039957939,
 'HasAns_f1': 68.10270430738815,
 'HasAns_total': 951,
 'NoAns_exact': 0.0,
 'NoAns_f1': 0.0,
 'NoAns_total': 951,
 'best_exact': 70.55730809674027,
 'best_exact_thresh': 0.9450581669807434,
 'best_f1': 77.40731682626085,
 'best_f1_thresh': 0.9948939681053162}